In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
# 명시적 대기를 위해 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
# 상품 정보를 담는 클레스
class TourInfo:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    area  = ''
    link  = ''
    img   = ''
    contents = ''
    # 생성자
    def __init__(self, title, price, area, link, img, contents=None ):
        self.title = title
        self.price = price
        self.area  = area
        self.link  = link
        self.img   = img
        self.contents = contents

In [3]:
# 디비 처리, 연결, 해제, 검색어 가져오기, 데이터 삽입
import pymysql as my

class DBHelper:
    '''
    맴버변수 : 커넥션 
    '''
    conn = None
    '''
    생성자
    '''
    def __init__(self):
        self.db_init()
    '''
    맴버 함수
    '''
    def db_init(self):
        self.conn = my.connect(
                        host='localhost',
                        user='root',
                        password='1234',
                        db='pythondb',
                        charset='utf8',
                        cursorclass=my.cursors.DictCursor )
    
    def db_free(self):
        if self.conn:
            self.conn.close()

    # 검색 키워드 가져오기 => 웹에서 검색
    def db_selectKeyword(self):
        # 커서 오픈
        # with => 닫기를 처리를 자동으로 처리해준다 => I/O 많이 사용
        rows = None
        with self.conn.cursor() as cursor:
            sql  = "select * from tbl_keyword;"
            cursor.execute(sql)
            rows = cursor.fetchall()
            print(rows)
        return rows
        
    def db_insertCrawlingData(self, title, price, area, contents, keyword ):
        with self.conn.cursor() as cursor:
            sql = "insert into tbl_crawlingdata (title, price, area, contents, keyword) values( %s,%s,%s,%s,%s );"
            cursor.execute(sql, (title, price, area, contents, keyword) )
        self.conn.commit()

In [4]:
# 사전에 필요한 정보를 로드 => 디비혹스 쉘, 베치 파일에서 인자로 받아서 세팅
db       = DBHelper()
main_url = 'http://tour.interpark.com/' 
keyword  = '일본'
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

C:\Users\j\anaconda3\envs\multi\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 'db' is deprecated, use 'database'
  del sys.path[0]


In [5]:
# 윈도우용
driver = wd.Chrome(executable_path='./webdriver/chromedriver.exe')

In [6]:
# 사이트 접속 (get)
driver.get(main_url)
# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
# 수정할경우 => 뒤에 내용이 붙어버림 => .clear() -> send_keys('내용')
# 검색 버튼 클릭
driver.find_element_by_css_selector('button.search-btn').click()

In [7]:
# 잠시 대기 => 페이가 로드되고 나서 즉각적으로 데이터를 획득 하는 행위는 
# 명시적 대기 => 특정 요소가 로케이트(발결된때까지) 대기
try:
    element = WebDriverWait(driver, 10).until(
        # 지정한 한개 요소가 올라면 웨이트 종료
        EC.presence_of_element_located( (By.CLASS_NAME, 'oTravelBox') )
    )
except Exception as e:
    print( '오류 발생', e)
# 암묵적 대기 => DOM이 다 로드 될때까지 대기 하고 먼저 로드되면 바로 진행
# 요소를 찾을 특정 시간 동안 DOM 풀링을 지시 예를 들어 10 초이내 라로 
# 발견 되면 진행
driver.implicitly_wait( 10 )
# 절대기 대기 => time.sleep(10) -> 클라우드 페어(디도스 방어  솔류션)
# 더보기 눌러서 => 게시판 진입 
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

In [8]:
# searchModule.SetCategoryList(1, '') 스크립트 실행
# 16은 임시값, 게시물을 넘어갔을때 현상을 확인차
for page in range(1, 2):#16):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        #############################################################
        # 여러 사이트에서 정보를 수집할 경우 공통 정보 정의 단계 필요
        # 상품명, 코멘트, 기간1, 기간2, 가격, 평점, 썸네일, 링크(상품상세정보)
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나 하나 접근
        for li in boxItems:
            # 이미지를 링크값을 사용할것인가? 
            # 직접 다운로드 해서 우리 서버에 업로드(ftp) 할것인가?
            print( '썸네임', li.find_element_by_css_selector('img').get_attribute('src') )
            print( '링크', li.find_element_by_css_selector('a').get_attribute('onclick') )
            print( '상품명', li.find_element_by_css_selector('h5.proTit').text )
            print( '코멘트', li.find_element_by_css_selector('.proSub').text )
            print( '가격',   li.find_element_by_css_selector('.proPrice').text )
            area = ''
            for info in li.find_elements_by_css_selector('.info-row .proInfo'):
                print(  info.text )
            print('='*100)
            # 데이터 모음
            # li.find_elements_by_css_selector('.info-row .proInfo')[1].text
            # 데이터가 부족하거나 없을수도 있으므로 직접 인덱스로 표현은 위험성이 있음
            obj = TourInfo(  
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('.proPrice').text,
                li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
                li.find_element_by_css_selector('a').get_attribute('onclick'),
                li.find_element_by_css_selector('img').get_attribute('src')
            )
            tour_list.append( obj )
    except Exception as e1:
        print( '오류', e1 )

print( tour_list, len(tour_list) )
# 수집한 정보 개수를 루프 => 페이지 방문 => 콘텐츠 획득(상품상세정보) => 디비

1 페이지 이동
썸네임 http://tourimage.interpark.com/product/tour/00161/A60/280/A6013263_24_630.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A6013263','')
상품명 [오사카 4일] 돌아온 히트상품_최다후기_핵심관광+1일자유
코멘트 오사카 베스트셀러 NO.1
가격 899,000 원~
여행 기간 : 3박4일
출발 가능 기간 : 2021.07.01~2023.12.26
평점 9.2
431개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A60/280/A6013214_70_217.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A6013214','')
상품명 [북해도 4일]수퍼콤비 도야&노보리베츠 온천2박+삿포로시내 1박_티웨이항공
코멘트 부모님께 강력추천/온천2박/3대게 뷔페 포함
가격 799,000 원~
여행 기간 : 3박4일
출발 가능 기간 : 2021.07.01~2021.10.31
평점 9.3
426개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/B60/280/B6011678_8_130.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=B6011678','')
상품명 [오사카 자유3일] 일본여행의 정석_아시아나항공
코멘트 오사카 자유여행
가격 689,000 원~
여행 기간 : 2박3일
출발 가능 기간 : 2021.07.01~2021.12.31
평점 9.7
60개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00

In [9]:
tour_list[0].title

'[오사카 4일] 돌아온 히트상품_최다후기_핵심관광+1일자유'

In [10]:
# 수집한 정보 개수를 루프 => 페이지 방문 => 콘텐츠 획득(상품상세정보) => 디비
for tour in tour_list:
    # tour => TourInfo
    print( type(tour) )
    # 링크 데이터에서 실데이터 획득
    # 분해
    arr = tour.link.split(',')
    if arr:
        # 대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        # 슬라이싱 => 앞에 ', 뒤에 ' 제거
        detail_url = link[1:-1]
        # 상세 페이지 이동 : URL 값이 완성된 형태인지 확인 (http~)
        driver.get( detail_url ) # 정확한 url인지 확인 차 들어가봄
        time.sleep(2)
        # pip install bs4
        # 혖재 페이지를 beautifulsoup 의 DOM으로 구성
        soup = bs( driver.page_source, 'html.parser')
        # 현제 상세 정보 페이지에서 스케줄 정보 획득
        data = soup.select('.tip-cover')
        print( type(data), len(data), type(data[0].contents)  )
        # 
        # 데이터 sum
        data[0].contents = 'test' # me
        content_final = ''
        for c in data[0].contents:
            content_final += str(c)
        
        # html 콘첸츠 데이터 전처리 (디비에 입력 가능토록)
        import re
        content_final   = re.sub("'", '"', content_final)
        content_final   = re.sub(re.compile(r'\r\n|\r|\n|\n\r+'), '', content_final)

        print( content_final )
        # 콘텐츠 내용에 따라 전처리 => data[0].contents
        db.db_insertCrawlingData(
            tour.title,
            tour.price[:-1],
            tour.area.replace('출발 가능 기간 : ',''),
            content_final,
            keyword
        )

<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test
<class '__main__.TourInfo'>
<class 'bs4.element.ResultSet'> 1 <class 'list'>
test


In [11]:
# 종료
driver.close()
driver.quit()
# import sys
# sys.exit()

In [12]:

# # DB test        
# if __name__=='__main__':
#     db = DBHelper()
#     print( db.db_selectKeyword() )
#     print( db.db_insertCrawlingData('1','2','3','4','5') )
#     db.db_free()
